## Package imports, global variables and functions

In [1]:
import mysql.connector
import json
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 15]

host = "localhost"
un = "root"
pw=""
db_name = "yelp"

def mysql_result_to_df(result, cursor):
    field_names = [i[0] for i in mycursor.description]
    return pd.DataFrame(myresult, columns=field_names)

def df_to_csv(df, fp, ext=".csv", na_rep=""):
    try:
        df.to_csv(fp + ext, encoding="utf-8", header = True,\
            doublequote = True, sep=",", index=False, na_rep=na_rep)
    except Exception as e:
        print("Error: {}".format(str(e)))

fp = "C:/Users/Tolis/Documents/Data Analytics Cource/CKME136 X10/Project/data/final/summary"

## Connect to yelp db and load table

In [2]:
mydb = mysql.connector.connect(
  host=host,
  user=un,
  passwd=pw,
  database=db_name
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM photo")

myresult = mycursor.fetchall()

photo_df = mysql_result_to_df(myresult, mycursor)

#Replace blank strings or objects {} to null
photo_df = photo_df.replace(r'^\s*$', np.nan, regex=True)
photo_df = photo_df.replace(r'^{}*$', np.nan, regex=True)

mycursor.close()
mydb.close()

In [3]:
print(photo_df[0:10])

                                             caption                photo_id  \
0                                                NaN  --3uF9gf2WttniseRLlI6g   
1                                        Edible menu  --4v2HCSXi092RwfidxlMg   
2                                                NaN  --5FyXiUudlvrJyUCvpZJA   
3                                                NaN  --6kOAkha3ULjhhw32KN1w   
4                                                NaN  --8aLa0rf2gfOjA6kCC3WQ   
5         From website. Desperately want this...now!  --Be9sfiHG00D6eWN3jcpg   
6                                                NaN  --CI0N0I4Wy1D-EPUJCceg   
7  This was the winner--fish and veggies were exc...  --CTOdfZ8W5_5n8LePlIpg   
8                                                NaN  --Dbsrqja2a_LtBreU295g   
9                                    Crab fried rice  --DpaHUw76HtjHogXfLXnA   

              business_id   label  
0  yYHdC09T718YaF0KYb6rpg  inside  
1  umXvdus9LbC6oxtLdXelFQ    food  
2  aqrTv6Qi

## Qualitative summaries
<p>This table does not have any numeric cols</p>

In [4]:
#Description for qualitative
df_description_cat = photo_df.describe(include=[np.object])
print(df_description_cat)

         caption                photo_id             business_id   label
count      92150                  200000                  200000  200000
unique     71512                  200000                   30488       5
top     Interior  KxXyyXZr5wvdrCZeTt_L5Q  RESDUcs7fIiihp38-d6_6g    food
freq         458                       1                     823  114874


In [5]:
#Row and col counts.
row_count = photo_df.shape[0]
col_count = photo_df.shape[1]
print(photo_df.shape)

(200000, 4)


In [6]:
#Summary of NULLS
#Will need to drop fields that have a Null Percent > .5
#I will then fill the remaining null values with their field's average.
null_columns=photo_df.columns[photo_df.isnull().any()]
df_description_nulls=photo_df[null_columns].isnull().sum()
df_description_nulls = pd.DataFrame(df_description_nulls, columns=["Null Count"])
df_description_nulls["Null Percent"] = df_description_nulls["Null Count"].apply(lambda x: x/row_count)
print(df_description_nulls)

         Null Count  Null Percent
caption      107850       0.53925


# Plots

In [7]:
#Clean up and save description csv.
#The plots can be obtained via screenshot.

#To make sure field names make it in output csv
row_labels = df_description_nulls.index.values
df_description_nulls.insert (0, "Field", row_labels)
df_to_csv(df_description_nulls, fp + "/photo_desc_nulls", ext=".csv", na_rep="")


row_labels = df_description_cat.index.values
df_description_cat.insert (0, "Param", row_labels)
df_to_csv(df_description_cat, fp + "/photo_desc_cat", ext=".csv", na_rep="")


In [8]:
photo_df = None
photo_df_cat = None